In [4]:
from scipy.stats import qmc, norm
from Data import merged_df as df
from main import dates
import numpy as np
from datetime import timedelta
import math
from collections import defaultdict
from scipy.optimize import minimize

if 'Date' in df.columns:
    df.set_index('Date',inplace=True)
    
rf_nifty,rf_ftse,rf_gold=0.06,0.04,0.03

# Function to calculate the optimal hedge ratio (h1, h2, h3) for a single day
def find_hedge_ratio(payoff, a1, a2, a3):
    # Objective function to minimize the variance of the hedged portfolio
    def objective(h):
        val=payoff + h[0] * a1 + h[1] * a2 + h[2] * a3
        return np.var(val)

    # Initial guess for hedge ratios
    initial_guess = [0.0, 0.0, 0.0]

    # minimise the variance of the portfolio (discounted payoff on day i) + h1price_asset1 + h2price_asset2 + h3*price_asset3
    result = minimize(objective, initial_guess, method='SLSQP')

    # Return the optimal hedge ratios if optimization is successful, else return NaNs
    return result.x if result.success else [np.nan, np.nan, np.nan]


def generate_RVs(num, corr_mat):
    sobol_sampler = qmc.Sobol(d=3, scramble=True)
    sobol_samples = sobol_sampler.random_base2(m=7)
    sobol_samples = np.clip(sobol_samples, 1e-10, 1 - 1e-10)
    
    random_variables = norm.ppf(sobol_samples)
    demeaned_random_variables = random_variables - np.mean(random_variables, axis=0)
    std_devs = np.std(demeaned_random_variables, axis=0)
    std_devs[std_devs == 0] = 1e-10
    standardized_random_variables = demeaned_random_variables / std_devs

    cholesky_matrix = np.linalg.cholesky(corr_mat)
    correlated_sobol_variables = np.dot(standardized_random_variables, cholesky_matrix.T)
    
    antithetic_variates = -correlated_sobol_variables[:num, :]
    return np.vstack([correlated_sobol_variables[:num, :], antithetic_variates])


def simulate_stock_prices_multi(initial_prices,  volatilities, days, corr_mat, num_simulations=10000,mean_returns=[rf_nifty,rf_ftse,rf_gold]):
    num_stocks = len(initial_prices)
    simulated_prices = np.zeros((days + 1, num_stocks, num_simulations))
    simulated_prices[0] = np.array(initial_prices)[:, None]

    dt = 1 / 252

    for sim in range(num_simulations):
        random_numbers = generate_RVs(days, corr_mat)
        for stock in range(num_stocks):
            for t in range(1, days + 1):
                simulated_prices[t, stock, sim] = simulated_prices[t - 1, stock, sim] * np.exp(
                    (mean_returns[stock] - 0.5 * volatilities[stock]**2) * dt + volatilities[stock] * np.sqrt(dt) * random_numbers[t - 1, stock]
                )
                
    return simulated_prices


In [16]:
import pandas as pd
df_filtered=df[df.index[0]:df.index[0]+timedelta(days=365.25*5)]
df_test=df[df.index[0]+timedelta(days=365.25*5+1):]
returns = df_filtered[['open_nifty','open_ftse','open_gold']].pct_change().dropna()
v_gold=df_filtered[df.index[0]+timedelta(days=365.25*2):df.index[0]+timedelta(days=365.25*5)]['open_gold'].pct_change().dropna().std()* np.sqrt(252)
v_nifty=df_filtered[df.index[0]+timedelta(days=365.25*2):df.index[0]+timedelta(days=365.25*5)]['open_nifty'].pct_change().dropna().std()* np.sqrt(252)
v_ftse=df_filtered[df.index[0]+timedelta(days=365.25*2):df.index[0]+timedelta(days=365.25*5)]['open_ftse'].pct_change().dropna().std()* np.sqrt(252)
corr_mat = returns.corr().to_numpy()

def top(d):
    sorted_items = sorted(d.items(), key=lambda x: x[1], reverse=True)
    first = sorted_items[0][0]
    second = sorted_items[1][0]
    third = sorted_items[2][0]
    
    return first,second,third


prod_end=0
num_simulations=10000
init_cap=100
day_start=pd.Timestamp('2024-10-15') ###random default init
day_end = pd.Timestamp('2024-10-15') ###random default init
prices_list=[]

n0,ftse0,gold0=0,0,0 ##dummy init

contribution=defaultdict(list)

hedge_df = pd.DataFrame()

for quarter, group in df_test.groupby(pd.Grouper(freq='Q')):
    if(prod_end==0):
        day_start=group.index[0]
        n0=group['open_nifty'][0]
        ftse0=group['open_ftse'][0]
        gold0=group['open_gold'][0]

    sim=simulate_stock_prices_multi([n0,ftse0,gold0],[v_nifty,v_ftse,v_gold],group.shape[0]-1,corr_mat)
    
    p1,p2,p3=sim[:,0,:],sim[:,1,:],sim[:,2,:] ###day x sim_no.
    p1 = pd.DataFrame(p1)
    p2 = pd.DataFrame(p2)
    p3 = pd.DataFrame(p3)
    
    return_stock1 = (1+p1.pct_change().dropna()).prod()-1
    return_stock2 = (1+p2.pct_change().dropna()).prod()-1
    return_stock3 = (1+p3.pct_change().dropna()).prod()-1

    # Determine the best and second-best performing stocks for each simulation
    returns = np.array([return_stock1, return_stock2, return_stock3])

    val1 = np.zeros(num_simulations)
    val2 = np.zeros(num_simulations)

    for simu in range(num_simulations):
        sorted_indices = np.argsort(returns[:, simu])[::-1]  # Sort returns in descending order
        best_stock = sorted_indices[0]
        second_best_stock = sorted_indices[1]

        # Assign val1 and val2 based on the best and second-best stocks
        val1[simu] = returns[best_stock, simu]
        val2[simu] = returns[second_best_stock, simu]
    
    hedge_ratios_list = []

    for day in range(0, group.shape[0]): ######DO FROM HERE

        dt=(group.index[-1]-group.index[day]).days
        
        payoff = init_cap*(0.67 * (1+val1) + 0.33 * (1+val2)) * np.exp(-rf_gold * dt/365)

        a1 = p1.iloc[day, :]
        a2 = p2.iloc[day, :]
        a3 = p3.iloc[day, :]

        hedge_ratios = find_hedge_ratio(payoff, a1, a2, a3)

        # Store the hedge ratios in a list for each day
        hedge_ratios_list.append({
            'Date':group.index[day],
            'Hedge_Ratio_Stock1': hedge_ratios[0],
            'Hedge_Ratio_Stock2': hedge_ratios[1],
            'Hedge_Ratio_Stock3': hedge_ratios[2]
        })

    hedge_ratios_df = pd.DataFrame(hedge_ratios_list)
    
    hedge_df = pd.concat([hedge_df, hedge_ratios_df], ignore_index=True)

    sim=sim.mean(axis=2)
    
    sim_df = pd.DataFrame(sim, columns=['nifty', 'ftse', 'gold'])
    
    simi={'nifty':(1+(sim_df['nifty'].pct_change().dropna())).prod(),
        'ftse':(1+(sim_df['ftse'].pct_change().dropna())).prod(),
        'gold':(1+(sim_df['gold'].pct_change().dropna())).prod()}
    f1,s1,t1=top(simi)
    
    contribution[f1].append(0.67)
    contribution[s1].append(0.33)
    contribution[t1].append(0.0)
    
    init_cap=0.67*init_cap*((1+(sim_df[f"{f1}"].pct_change().dropna())).prod()) + 0.33*init_cap*((1+(sim_df[f"{s1}"].pct_change().dropna())).prod())
    print('Portfolio value:',init_cap)
    
    n0,ftse0,gold0=sim_df['nifty'].iloc[-1],sim_df['ftse'].iloc[-1],sim_df['gold'].iloc[-1]
    
    prod_end+=1
    if(prod_end==12):
        day_end=group.index[-1]
        days=(day_end-day_start).days
        print(f"After {days} days PRODUCT PRICED!!!")
        prod_end=0
        average_dict = {key: sum(values) / len(values) for key, values in contribution.items()}
        cont_nifty=average_dict['nifty']
        cont_ftse=average_dict['ftse']
        cont_gold=average_dict['gold']
        
        price= init_cap*(cont_nifty*math.exp(-rf_nifty*(days/365))+ 
                        cont_ftse*math.exp(-rf_ftse*(days/365)) + cont_gold*math.exp(-rf_gold*(days/365)))
        prices_list.append([price,day_start,day_end])
        contribution.clear()
        init_cap=100
        df_filtered=df[quarter-timedelta(days=365.25*5):quarter]
        returns = df_filtered[['open_nifty','open_ftse','open_gold']].pct_change().dropna()
        corr_mat = returns.corr().to_numpy()
        v_gold=df[group.index[-1]-timedelta(days=365.25*3):group.index[-1]]['open_gold'].pct_change().dropna().std()* np.sqrt(252)
        v_nifty=df_filtered[group.index[-1]-timedelta(days=365.25*3):group.index[-1]]['open_nifty'].pct_change().dropna().std()* np.sqrt(252)
        v_ftse=df_filtered[group.index[-1]-timedelta(days=365.25*3):group.index[-1]]['open_ftse'].pct_change().dropna().std()* np.sqrt(252)


df_test.reset_index(inplace=True)
merged_df = pd.merge(df_test, hedge_df, on='Date', suffixes=('_df_test', '_hedge_df'))
delta_hedge = merged_df[['Hedge_Ratio_Stock1', 'Hedge_Ratio_Stock2', 'Hedge_Ratio_Stock3']].diff().fillna(0)
cash_change = -(delta_hedge.values * merged_df[['open_nifty', 'open_ftse', 'open_gold']].values).sum(axis=1)
merged_df['Cash_Position'] = cash_change.cumsum()
merged_df

Portfolio value: 100.4692358125821
Portfolio value: 101.24048382310781
Portfolio value: 102.02138585287196
Portfolio value: 102.83892446210135
Portfolio value: 103.6400751828419
Portfolio value: 104.44275882683374
Portfolio value: 105.24564292915846
Portfolio value: 106.10586282875303
Portfolio value: 106.94501574163543
Portfolio value: 107.74194572973269
Portfolio value: 108.55303865027386
Portfolio value: 109.42775401596424
After 1057 days PRODUCT PRICED!!!
Portfolio value: 100.46104685307691
Portfolio value: 100.9263815036426
Portfolio value: 101.38126050485951
Portfolio value: 101.90295441646805
Portfolio value: 102.38749387653581
Portfolio value: 102.88298978883236
Portfolio value: 103.41285633567328
Portfolio value: 103.94386624285126
Portfolio value: 104.46836060159544
Portfolio value: 104.97038690224741
Portfolio value: 105.4683513781712
Portfolio value: 105.98051712141485
After 1094 days PRODUCT PRICED!!!
Portfolio value: 100.26012848309546
Portfolio value: 100.51125729036806


,Date,int_nifty,int_ftse,int_gold,open_nifty,open_ftse,open_gold,nifty_vol,ftse_vol,gold_vol,...,delta_nifty,delta_ftse,delta_gold,Returns_nifty,Returns_ftse,Returns_gold,Hedge_Ratio_Stock1,Hedge_Ratio_Stock2,Hedge_Ratio_Stock3,Cash_Position
0,2005-11-08,0.051549,0.000515,0.000515,2463.650000,5460.799805,459.45,0.014987,0.009624,0.008961,...,-0.343305,-0.497274,-0.497105,0.018353,0.006859,0.005361,0.000000,0.000000,0.000000,-0.000000
1,2005-11-09,0.053169,0.000532,0.000532,2493.100000,5460.899902,460.60,0.014945,0.009622,0.008960,...,-0.338241,-0.497195,-0.497020,0.011954,0.000018,0.002503,0.005931,-0.003409,0.035865,-12.691088
2,2005-11-10,0.052069,0.000521,0.000521,2489.100000,5439.799805,466.90,0.014912,0.009291,0.009424,...,-0.341065,-0.497166,-0.497200,-0.001604,-0.003864,0.013678,-0.009851,0.000354,-0.009176,27.149999
3,2005-11-11,0.049074,0.000491,0.000491,2500.850000,5423.500000,466.90,0.014098,0.009032,0.008519,...,-0.341554,-0.497252,-0.497111,0.004721,-0.002996,0.000000,-0.014279,-0.002224,0.041528,28.535681
4,2005-11-14,0.049351,0.000494,0.000494,2548.550000,5465.100098,469.40,0.014751,0.009143,0.008596,...,-0.347373,-0.497266,-0.497119,0.019074,0.007670,0.005354,-0.009229,-0.003272,0.027438,28.008508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4540,2024-09-26,0.052583,0.000526,0.000526,25899.449219,8268.700195,2657.32,0.006117,0.005771,0.007003,...,-0.157109,-0.495611,-0.496331,-0.000849,-0.001702,0.000535,-0.001420,-0.002648,-0.008681,28.080615
4541,2024-09-27,0.045093,0.000451,0.000451,26005.400391,8284.900391,2669.50,0.006103,0.005768,0.007016,...,-0.193498,-0.496215,-0.496835,0.004091,0.001959,0.004584,-0.001095,-0.001930,-0.005877,6.182851
4542,2024-09-30,0.042976,0.000430,0.000430,26248.250000,8320.799805,2658.30,0.006300,0.005852,0.006845,...,-0.212217,-0.496435,-0.496909,0.009338,0.004333,-0.004196,-0.002376,-0.004167,-0.012936,77.203078
4543,2024-10-01,0.050536,0.000505,0.000505,26061.300781,8237.000000,2635.41,0.006596,0.006133,0.007198,...,-0.184829,-0.496009,-0.496554,-0.007122,-0.010071,-0.008611,0.000000,0.000000,0.000000,-53.141525


In [17]:
for price in prices_list:
    print(f"Price of product from {price[1].date()} to {price[2].date()} period is: {price[0]}")

Price of product from 2005-11-08 to 2008-09-30 period is: 93.78427888611517
Price of product from 2008-10-01 to 2011-09-30 period is: 92.20159817840796
Price of product from 2011-10-03 to 2014-09-30 period is: 88.97130247120607
Price of product from 2014-10-01 to 2017-09-29 period is: 95.39600334592019
Price of product from 2017-10-03 to 2020-09-30 period is: 95.92637060886136
Price of product from 2020-10-01 to 2023-09-29 period is: 93.37801232872442
